<div style = "text-align: left"><font size = 6 color = "#B22222" face = "verdana"><b>Helsinki Service Property Energy Analysis</b></font></div>
<div style = "text-align: left"><font size = 4 color = "#00008B" face = "verdana"><b>Exploratory data analysis</b></font></div>
<div style = "text-align: leftt"><font><i>Jerry & Mayen</i></font></div>
<div style = "text-align: leftt"><font>18th Feb 2023</font></div>

### Introduction

The API relates to energy consumption data of some Utility and  Service Properties in Helsinki

Nuuka open API provides a platform for getting information on the energy data of of Helsinki Utility and Service Properties. 
The endpoint is useful for understanding the energy consumption profile and other additional informations such as heating, 
water and cooling. Energy can be fetched on ```hourly```, ```daily``` and ```monthly``` basis.

The API requires no authentication. Precise energy data are generally available from 2015 onwards. 
The data is updated once a day, and the result is returned in JSON format. data are usually not 
validated hence may contain errors


In [2]:
import requests
import warnings
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime
import helper as hp

### Notebook settings 

In [3]:
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = None
pd.set_option("display.float_format", lambda x: '%.2f' % x)
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### List of properties

In [4]:
# decalre the url for the list of properties
list_url = 'https://helsinki-openapi.nuuka.cloud/api/v1.0/Property/List'

# call the function for getting the list of all properties from helper.py
property_list = hp.total_list_of_properties(list_url)

# print the total number of properties and the dataframe returned by the function 
print(f"number of properties:{property_list['propertyName'].nunique()}")
property_list.head()

number of properties:842


,locationName,propertyName,propertyCode
0,1000 Hakaniemen kauppahalli,1000 Hakaniemen kauppahalli,091-011-9902-0101
1,1001 Hietalahden kauppahalli,1001 Hietalahden kauppahalli,091-004-9902-0008
2,1002 Vanha kauppahalli,1002 Vanha kauppahalli,091-003-9906-0101
3,1037 Vuotalo,1037 Vuotalo,091-054-0179-0003
4,1507 Suutarilan monitoimitalo/ala-aste ja Lpk Seulanen,1507 Suutarilan monitoimitalo/ala-aste ja Lpk Seulanen,091-040-0024-0003


### Store the list of properties dataframe as json file

In [5]:
# convert the list of property dataframe to json format 
# json_data = property_list.to_json(orient='records')

# # specify the file name and write the JSON data to a file 
# with open('data.json', 'w') as json_file:
#     json_file.write(json_data)

### Get hourly, daily and  monthly consumption for the year 2021 and 2022 

# 2022

### Electricity

In [6]:
daily_elect_2022_df = hp.get_request_as_df(period='Daily',
record="LocationName",
search_string='1000 Hakaniemen kauppahalli',
reporting_group="Electricity",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=False,
version=1.0)

In [7]:
print(f"dataframe shape:{daily_elect_2022_df.shape}")
daily_elect_2022_df.head()

dataframe shape:(365, 5)


,timestamp,reportingGroup,locationName,value,unit
0,2022-01-01,Electricity,1000 Hakaniemen kauppahalli,20.35,kWh
1,2022-01-02,Electricity,1000 Hakaniemen kauppahalli,139.39,kWh
2,2022-01-03,Electricity,1000 Hakaniemen kauppahalli,1.18,kWh
3,2022-01-04,Electricity,1000 Hakaniemen kauppahalli,1.16,kWh
4,2022-01-05,Electricity,1000 Hakaniemen kauppahalli,22.10,kWh


### Water

In [8]:
hourly_water_2022_df = hp.get_request_as_df(period='Hourly',
record="LocationName",
search_string="1000 Hakaniemen kauppahalli",
reporting_group="Water",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=False,
version=1.0)

In [9]:
print(f"dataframe shape:{hourly_water_2022_df.shape}")
hourly_water_2022_df.head()

dataframe shape:(2, 5)


,timestamp,reportingGroup,locationName,value,unit
0,2022-09-08,Water,1000 Hakaniemen kauppahalli,7173,M3
1,2022-11-30,Water,1000 Hakaniemen kauppahalli,961,M3


### Heat

In [10]:
hourly_heat_2022_df = hp.get_request_as_df(period='Hourly',
record="LocationName",
search_string="1000 Hakaniemen kauppahalli",
reporting_group="Heat",
start_time="2022-01-01",
end_time="2022-12-31",
Normalization=True,
version=1.0)

In [11]:
print(f"dataframe shape:{hourly_heat_2022_df.shape}")
hourly_heat_2022_df.head()

dataframe shape:(8544, 5)


,timestamp,reportingGroup,locationName,value,unit
0,2022-01-01 00:00:00,Heat,1000 Hakaniemen kauppahalli,110.10,kWh
1,2022-01-01 01:00:00,Heat,1000 Hakaniemen kauppahalli,111.30,kWh
2,2022-01-01 02:00:00,Heat,1000 Hakaniemen kauppahalli,110.80,kWh
3,2022-01-01 03:00:00,Heat,1000 Hakaniemen kauppahalli,110.00,kWh
4,2022-01-01 04:00:00,Heat,1000 Hakaniemen kauppahalli,111.70,kWh


In [20]:
hourly_heat_2022_df.tail()

,timestamp,reportingGroup,locationName,value,unit
8539,2022-12-30 20:00:00,Heat,1000 Hakaniemen kauppahalli,78.50,kWh
8540,2022-12-30 21:00:00,Heat,1000 Hakaniemen kauppahalli,78.60,kWh
8541,2022-12-30 22:00:00,Heat,1000 Hakaniemen kauppahalli,80.00,kWh
8542,2022-12-30 23:00:00,Heat,1000 Hakaniemen kauppahalli,78.90,kWh
8543,2022-12-31 00:00:00,Heat,1000 Hakaniemen kauppahalli,79.20,kWh


In [12]:
import os
import pandas as pd
import requests
import json

In [14]:
df1 = pd.read_json('data.json')

# list of location names with space replacement
location_names = [location.replace(" ", "%20") for location in df1['locationName']]

# List to store URLs with valid JSON data
valid_urls = []

# Fetching data for each location
for location in location_names:
    #print()
    url = f"https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString={location}&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31"
    response = requests.get(url)
    data = response.json()
    if 'errorCode' not in data or data['errorCode'] != 'MissingSettingsException':
        valid_urls.append(url)


['https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1000%20Hakaniemen%20kauppahalli&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31', 'https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1001%20Hietalahden%20kauppahalli&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31', 'https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1002%20Vanha%20kauppahalli&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31', 'https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1037%20Vuotalo&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31', 'https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1507%20Suutarilan%20monitoimitalo/ala-aste%20ja%20Lpk%20

In [15]:
# Creating a folder to store the JSON files
folder_name = "energy_data"
os.makedirs(folder_name, exist_ok=True)

In [16]:
# # Save data from each valid URL as a separate JSON file in the folder
for index, url in enumerate(valid_urls):
    response = requests.get(url)
    data = response.json()
    filename = f"data_{index}.json"
    file_path = os.path.join(folder_name, filename)
    with open(file_path, 'w') as file:
        json.dump(data, file)
    print(f"Data saved for URL: {url} (filename: {filename})")

print("Data saved successfully.")


Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1000%20Hakaniemen%20kauppahalli&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_0.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1001%20Hietalahden%20kauppahalli&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_1.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1002%20Vanha%20kauppahalli&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_2.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1037%20Vuotalo&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_3.json)
Data 

Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1637%20Lp%20Kesanto&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_32.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1644%20Työväen%20asuntomuseo&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_33.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1648%20Kanneltalo,%20monitoimitalo&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_34.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1650%20Hoplaxskolan/Dh%20Sockenstugan&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: 

Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1805%20Konalan%20ala-asteen%20koulu,%20NT&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_64.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1812%20Kulttuurikeskus%20Stoa&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_65.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1833%20Lpk%20Kotilo%20(disabled)&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_66.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1838%20Lp%20Piika&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename:

Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=2186%20Lpk%20Ilomäki,%20Lp%20Kiiltotähti&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_96.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=2187%20Viikin%20nuorisotalo&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_97.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=2194%20Lp%20Isonneva&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_98.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=2195%20Lpk%20Laajasuo&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_99.js

Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4017%20Lpk%20Eira&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_128.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4019%20Lp%20Linja,%20Ilolan%20puisto&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_129.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4020%20Lpk%20Pehtoori&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_130.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4022%20Taivallahden%20peruskoulu&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_131.

...

Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4489%20Lpk%20The%20Little%20English%20Kindergarten%20Eläintarha&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_449.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4490%20Herttoniemenrannan%20ala-aste&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_450.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4491%20Palvelutalo%20Kivitasku&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_451.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4492%20Lpk%20Ruusu&ReportingGroup=Electricity&StartTime=2019-01-0

Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4565%20Kumpulan%20maauimala/PO&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_481.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4567%20Käpylän%20liikuntapuisto/PO&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_482.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4570%20Lassilan%20liikuntapuisto/PO&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_483.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4581%20Puotilan%20liikuntapuisto/PO&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=201

Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4723%20Mustikkamaanpolku%2010%20(disabled)&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_513.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4724%20Lpk%20Meritähti&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_514.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=6449%20Lpk%20Ilves/Hillerikuja%201b&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_515.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=6450%20Lpk%20Immola&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filenam

Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4331%20Siltavuorenranta%20väestönsuoja/PO&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_544.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4345%20Strömberginpuisto%20väestönsuoja/PO&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_545.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4384%20Torkkelinmäki%20väestönsuoja/PO&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_546.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4397%20Tähtitorni%201%20&%202%20väestönsuoja/PO&ReportingGroup=Electricit

Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4238%20Itäkeskuksen%20uimahalli/PO&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_575.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=2416%20Lp%20Trumpetti/Uusi%20korvaava&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_576.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4656%20Lpk%20Borgströminmäki&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31 (filename: data_577.json)
Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=2287%20Lpk%20Lasten%20Kartano/Malminkartanon%20nuorisotalo,%2067659%20Päiväkotirakennus&Reporti

In [17]:
folder_name = "energy_data"

# List all JSON files in the folder
json_files = [file for file in os.listdir(folder_name) if file.endswith(".json")]

# List to store merged data
merged_data = []

# Read and merge data from each JSON file
for file in json_files:
    file_path = os.path.join(folder_name, file)
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
        merged_data.extend(data)

# Save the merged data as a single JSON file
output_file = "merged_data.json"
with open(output_file, 'w') as file:
    json.dump(merged_data, file)

print(f"Merged data saved successfully as {output_file}.")

Merged data saved successfully as merged_data.json.


### Load 2019 electricity consumption data

In [23]:
# Read the JSON file into a pandas DataFrame
df = pd.read_json('merged_data.json')
print(f"the shape of the dataframe is: {df.shape}")
df

the shape of the dataframe is: (205322, 5)


,timestamp,reportingGroup,locationName,value,unit
0,2019-01-08,Electricity,1000 Hakaniemen kauppahalli,0.00,kWh
1,2019-01-09,Electricity,1000 Hakaniemen kauppahalli,0.00,kWh
2,2019-01-10,Electricity,1000 Hakaniemen kauppahalli,0.00,kWh
3,2019-01-11,Electricity,1000 Hakaniemen kauppahalli,20.10,kWh
4,2019-01-12,Electricity,1000 Hakaniemen kauppahalli,30.56,kWh


In [21]:
df.tail()

,timestamp,reportingGroup,locationName,value,unit
205317,2019-12-27,Electricity,2195 Lpk Laajasuo,205.29,kWh
205318,2019-12-28,Electricity,2195 Lpk Laajasuo,223.04,kWh
205319,2019-12-29,Electricity,2195 Lpk Laajasuo,180.00,kWh
205320,2019-12-30,Electricity,2195 Lpk Laajasuo,172.12,kWh
205321,2019-12-31,Electricity,2195 Lpk Laajasuo,170.93,kWh


In [29]:
get_daily_energy_for_a_specified_year('2020-01-01', '2020-12-31')

Data saved for URL: https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=6829%20Helsingin%20Kalasataman%20Kymppi%20Koy,%2066440%20Kymp-talo%20rak%20001&ReportingGroup=Electricity&StartTime=2020-01-01&EndTime=2020-12-31 (filename: data_582.json)
Data saved successfully.
the shape of the dataframe is: (205322, 5)


,timestamp,reportingGroup,locationName,value,unit
0,2019-01-08,Electricity,1000 Hakaniemen kauppahalli,0.00,kWh
1,2019-01-09,Electricity,1000 Hakaniemen kauppahalli,0.00,kWh
2,2019-01-10,Electricity,1000 Hakaniemen kauppahalli,0.00,kWh
3,2019-01-11,Electricity,1000 Hakaniemen kauppahalli,20.10,kWh
4,2019-01-12,Electricity,1000 Hakaniemen kauppahalli,30.56,kWh
